In [ ]:
import sys
sys.path.append('../..')

In [24]:
from pathlib import Path

data_root = Path('/Users/knpob/Dev/data/3dMD/jump')

## Clean meshes

In [25]:
from glob import glob

mesh_files = sorted(glob(str(data_root / 'meshes' / '*.obj')))
mesh_files

['/Users/knpob/Dev/data/3dMD/jump/meshes/S6_nude_jump120_0309_0372.000001.obj',
 '/Users/knpob/Dev/data/3dMD/jump/meshes/S6_nude_jump120_0309_0372.000002.obj',
 '/Users/knpob/Dev/data/3dMD/jump/meshes/S6_nude_jump120_0309_0372.000003.obj',
 '/Users/knpob/Dev/data/3dMD/jump/meshes/S6_nude_jump120_0309_0372.000004.obj',
 '/Users/knpob/Dev/data/3dMD/jump/meshes/S6_nude_jump120_0309_0372.000005.obj',
 '/Users/knpob/Dev/data/3dMD/jump/meshes/S6_nude_jump120_0309_0372.000006.obj',
 '/Users/knpob/Dev/data/3dMD/jump/meshes/S6_nude_jump120_0309_0372.000007.obj',
 '/Users/knpob/Dev/data/3dMD/jump/meshes/S6_nude_jump120_0309_0372.000008.obj',
 '/Users/knpob/Dev/data/3dMD/jump/meshes/S6_nude_jump120_0309_0372.000009.obj',
 '/Users/knpob/Dev/data/3dMD/jump/meshes/S6_nude_jump120_0309_0372.000010.obj',
 '/Users/knpob/Dev/data/3dMD/jump/meshes/S6_nude_jump120_0309_0372.000011.obj',
 '/Users/knpob/Dev/data/3dMD/jump/meshes/S6_nude_jump120_0309_0372.000012.obj',
 '/Users/knpob/Dev/data/3dMD/jump/meshes

In [22]:
import os
from tqdm import tqdm
import pyvista as pv

os.makedirs(data_root / 'clean', exist_ok=True)

for mesh_file in tqdm(mesh_files, desc='Processing meshes'):
    mesh_clean = pv.read(mesh_file).clean().split_bodies()[0].extract_surface().triangulate()
    mesh_clean.save(str(data_root / 'clean' / Path(mesh_file).stem) + '.ply')

Processing meshes:   0%|          | 0/64 [00:00<?, ?it/s]

Processing meshes: 100%|██████████| 64/64 [00:09<00:00,  6.48it/s]


## Simplify meshes

In [42]:
import pymeshlab as ml

def mesh_simplify(input_mesh_path, output_mesh_path, target_vertex_count):
    ms = ml.MeshSet()
    ms.load_new_mesh(input_mesh_path)
    m = ms.current_mesh()

    # clean-up in case of input has duplicate/unreferenced vertices
    ms.meshing_remove_duplicate_vertices()
    ms.meshing_remove_unreferenced_vertices()

    # initial estimation number of faces, designed to be superior than the teorical value
    numFaces = 100 + ms.current_mesh().face_number() - (ms.current_mesh().vertex_number() - target_vertex_count)

    # iteratively simplify the mesh
    while (ms.current_mesh().vertex_number() > target_vertex_count):
        ms.meshing_decimation_quadric_edge_collapse(targetfacenum=numFaces, preservenormal=True)
        numFaces = numFaces - (ms.current_mesh().vertex_number() - target_vertex_count)

    m = ms.current_mesh()
    ms.save_current_mesh(output_mesh_path)#, save_textures=False)

In [26]:
mesh_clean_files = sorted(glob(str(data_root / 'clean' / '*.ply')))
mesh_clean_files

['/Users/knpob/Dev/data/3dMD/jump/clean/S6_nude_jump120_0309_0372.000001.ply',
 '/Users/knpob/Dev/data/3dMD/jump/clean/S6_nude_jump120_0309_0372.000002.ply',
 '/Users/knpob/Dev/data/3dMD/jump/clean/S6_nude_jump120_0309_0372.000003.ply',
 '/Users/knpob/Dev/data/3dMD/jump/clean/S6_nude_jump120_0309_0372.000004.ply',
 '/Users/knpob/Dev/data/3dMD/jump/clean/S6_nude_jump120_0309_0372.000005.ply',
 '/Users/knpob/Dev/data/3dMD/jump/clean/S6_nude_jump120_0309_0372.000006.ply',
 '/Users/knpob/Dev/data/3dMD/jump/clean/S6_nude_jump120_0309_0372.000007.ply',
 '/Users/knpob/Dev/data/3dMD/jump/clean/S6_nude_jump120_0309_0372.000008.ply',
 '/Users/knpob/Dev/data/3dMD/jump/clean/S6_nude_jump120_0309_0372.000009.ply',
 '/Users/knpob/Dev/data/3dMD/jump/clean/S6_nude_jump120_0309_0372.000010.ply',
 '/Users/knpob/Dev/data/3dMD/jump/clean/S6_nude_jump120_0309_0372.000011.ply',
 '/Users/knpob/Dev/data/3dMD/jump/clean/S6_nude_jump120_0309_0372.000012.ply',
 '/Users/knpob/Dev/data/3dMD/jump/clean/S6_nude_jump

In [44]:
import os
from tqdm import tqdm

os.makedirs(data_root / 'ply', exist_ok=True)

for mesh_file in tqdm(mesh_clean_files, desc='Processing meshes'):
    mesh_simplify(
        input_mesh_path=mesh_file,
        output_mesh_path=str(data_root / 'ply' / Path(mesh_file).stem) + '.ply',
        target_vertex_count=5000,
    )

Processing meshes: 100%|██████████| 64/64 [01:12<00:00,  1.14s/it]


In [45]:
pv.read(str(data_root / 'ply' / Path(mesh_file).stem) + '.ply').plot()

Widget(value='<iframe src="http://localhost:55786/index.html?ui=P_0x120bc5570_1&reconnect=auto" class="pyvista…

In [50]:
import shutil

# remove the cleaned meshes directory
shutil.rmtree(data_root / 'clean')